In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import json
import os

In [17]:
img = cv2.imread('masks/6_2.png')

imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(imgray, 127, 255, 0)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img, contours, -1, (0,255,0), 3)

# cv2.imshow(annotation['filename'], img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [20]:
# Create a usable list of points from the contours
sub_contours = []
shrinked_list = []

for contour in contours:
    sub_contour_points = []
    for contour_points in contour:
        sub_contour_points.append(contour_points[0])
    sub_contours.append(sub_contour_points)
    
    # Shrink the list of contours 
    shrinked_sub_list = []

    for i in range(0, len(sub_contour_points)):
        if i % 5 == 0: shrinked_sub_list.append(sub_contour_points[i])
    shrinked_list.append(shrinked_sub_list)
    
# print(sub_contours)
    
img = cv2.imread('masks/6_2.png')
for a in shrinked_list:
    a = np.array(a)
    red = [0,0,255]

    cv2.polylines(img, [a], True, red, 1)

# Show
cv2.imshow('image', img)

cv2.waitKey(0)
cv2.destroyAllWindows()
    


In [21]:
def find_contours(img):
    # Grayscale image 
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image
    ret, thresh = cv2.threshold(imgray, 127, 255, 0)

    # Find the contours
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Return an empty array if it did not find any contours
    if len(contours) == 0:
        return np.array([])
    
    # Create a usable list of points from the contours
    sub_contours = []
    shrinked_list = []

    for contour in contours:
        sub_contour_points = []
        for contour_points in contour:
            sub_contour_points.append(contour_points[0])
        sub_contours.append(sub_contour_points)

        # Shrink the list of contours 
        shrinked_sub_list = []

        for i in range(0, len(sub_contour_points)):
            if i % 5 == 0: shrinked_sub_list.append(sub_contour_points[i])
        shrinked_list.append(shrinked_sub_list)

    return np.array(shrinked_list)

In [23]:
# Create a usable list of points from the contours
img = cv2.imread('masks/6_2.png')
for a in find_contours(img):
    a = np.array(a)
    red = [0,0,255]

    cv2.polylines(img, [a], True, red, 1)

# Show
cv2.imshow('image', img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Testing contour on a mask
# Create a named colour
img = cv2.imread('masks/6_2.png')
contours = find_contours(img)

red = [0,0,255]

cv2.polylines(img, [contours], True, red, 3)

# Show
cv2.imshow('image', img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:
def get_class(filename):
    num = filename.split('_')[1].split('.')[0]
    
    if num == "1":
        return "one"
    elif num == "2":
        return "two"
    elif num == "3":
        return "three"
    elif num == "4":
        return "four"
    elif num == "5":
        return "five"
    elif num == "6":
        return "six"

In [24]:
class Mask(object):
    def __init__(self, file):
        self.name = file.split('/')[2]
        self.classification = get_class(self.name)
        self.contours = find_contours(cv2.imread(file))
        

class Image(object):
    def __init__(self, file):
        self.image = cv2.imread(file)
        self.name = file.split('/')[2]
        self.masks = self.get_masks("./masks/" + file.split('/')[2].split('.')[0] + "*.png")
        
    def get_masks(self, path):
        masks = []
        for file in glob.glob(path):
            masks.append(Mask(file))
            
        return masks
        

In [32]:
# Save the points to a json file
images = []

for file in glob.glob("./images/*.png"):
    images.append(Image(file))
    
data = {}

for image in images:
    regions = []
    for mask in image.masks:
        region = {}
        region['name'] = mask.name
        region['type'] = "polyline"
        region['class'] = mask.classification
        x_points = []
        y_points = []
        for points in mask.contours:
            points = np.array(points)
            print(points)
            x_points.append([points.transpose()[0].tolist()] if len(points) > 0 else [])
            y_points.append([points.transpose()[1].tolist()] if len(points) > 0 else [])
        region['all_points_x'] = x_points
        region['all_points_y'] = y_points
        
        regions.append(region)
    
    image_information = {}
    height, width, _ = image.image.shape
    image_information['filename'] = image.name
    image_information['width'] = width
    image_information['height'] = height
    image_information['regions'] = regions
    
    data['image_' + image.name] = image_information
    
with open('annotations.json', 'w') as json_file:
    json.dump(data, json_file)

[[339 263]
 [336 268]
 [334 274]
 [331 279]
 [329 285]
 [326 290]
 [324 296]
 [321 301]
 [319 307]
 [316 312]
 [314 318]
 [311 323]
 [309 329]
 [306 334]
 [304 340]
 [301 345]
 [301 352]
 [304 358]
 [306 363]
 [309 367]
 [315 370]
 [321 372]
 [323 367]
 [326 362]
 [328 356]
 [331 351]
 [334 348]
 [340 350]
 [345 353]
 [351 355]
 [358 356]
 [360 351]
 [363 345]
 [365 340]
 [368 334]
 [370 329]
 [373 323]
 [375 318]
 [378 312]
 [380 307]
 [373 303]
 [368 301]
 [362 298]
 [357 296]
 [358 290]
 [361 285]
 [363 279]
 [366 274]
 [365 269]
 [360 266]]
[[380 286]
 [377 292]
 [375 300]
 [377 305]
 [379 311]
 [376 316]
 [374 322]
 [371 327]
 [369 333]
 [366 338]
 [364 344]
 [361 349]
 [359 355]
 [353 360]
 [354 367]
 [360 371]
 [367 373]
 [374 374]
 [376 367]
 [379 360]
 [381 352]
 [384 346]
 [386 338]
 [389 332]
 [391 324]
 [394 317]
 [396 310]
 [399 303]
 [401 296]
 [394 292]
 [387 288]]
[[162 269]
 [159 275]
 [165 279]
 [172 283]
 [179 288]
 [186 292]
 [193 297]
 [199 300]
 [207 306]
 [213 30

In [ ]:
#For testing

annotations = json.load(open(os.path.join("annotations.json")))
annotations = list(annotations.values())

for annotation in annotations:
    img = cv2.imread("./images/" + annotation['filename'])

    for region in annotation['regions']:    
        points = np.array([region['all_points_x'], region['all_points_y']]).transpose()

        red = [0,0,255]

        cv2.polylines(img, [points], True, red, 2)

    # Show
    cv2.imshow(annotation['filename'], img)

cv2.waitKey(0)
cv2.destroyAllWindows()
    